In [8]:
import pandas as pd


In [10]:
dataset = pd.read_csv(r'D:\Documentos\UFCG\IA\ProjetoPredicao\predicao-precos\true_car_listings.csv')

In [11]:
print(dataset)

        Price  Year  Mileage               City State                Vin  \
0        8995  2014    35725            El Paso    TX  19VDE2E53EE000083   
1       10888  2013    19606   Long Island City    NY  19VDE1F52DE012636   
2        8995  2013    48851            El Paso    TX  19VDE2E52DE000025   
3       10999  2014    39922            Windsor    CO  19VDE1F71EE003817   
4       14799  2016    22142             Lindon    UT  19UDE2F32GA001284   
5        7989  2012   105246              Miami    FL  JH4CU2F83CC019895   
6       14490  2014    34032          Greatneck    NY  JH4CU2F84EC002686   
7       13995  2013    32384        West Jordan    UT  JH4CU2F64DC006203   
8       10495  2013    57596          Waterbury    CT  19VDE2E50DE000234   
9        9995  2013    63887            El Paso    TX  19VDE1F50DE010450   
10      12921  2012    58550              Boise    ID  JH4CU2F44CC003220   
11      12000  2013    40527   Long Island City    NY  19VDE1F38DE020867   
12       775